In [6]:
import sys
import os
# Add the project root directory to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

from pos.database import Database
import pandas as pd
from pprint import pprint
from collections import defaultdict
from datetime import datetime

db = Database()


--> Database connection established successfully!


In [7]:
data = db.fetch_all_sales()

# Get daily sales
daily_sales = defaultdict(float)
for sale in data:
    date = sale["date"].strftime('%Y-%m-%d')
    daily_sales[date] += float(sale["total_amount"])

daily_sale_data = [{"date": datetime.strptime(date, '%Y-%m-%d'), "total_amount": amount} for date, amount in daily_sales.items()]


In [8]:
daily_sale_data

[{'date': datetime.datetime(2024, 11, 23, 0, 0), 'total_amount': 40.0},
 {'date': datetime.datetime(2018, 5, 11, 0, 0), 'total_amount': 357.98},
 {'date': datetime.datetime(2014, 1, 12, 0, 0), 'total_amount': 811.02},
 {'date': datetime.datetime(2016, 8, 4, 0, 0), 'total_amount': 429.74},
 {'date': datetime.datetime(2019, 5, 8, 0, 0), 'total_amount': 762.98},
 {'date': datetime.datetime(2019, 11, 9, 0, 0), 'total_amount': 755.22},
 {'date': datetime.datetime(2014, 12, 11, 0, 0), 'total_amount': 117.46},
 {'date': datetime.datetime(2021, 3, 9, 0, 0), 'total_amount': 142.08},
 {'date': datetime.datetime(2014, 8, 12, 0, 0), 'total_amount': 814.26},
 {'date': datetime.datetime(2017, 10, 9, 0, 0), 'total_amount': 949.9},
 {'date': datetime.datetime(2017, 6, 2, 0, 0), 'total_amount': 638.18},
 {'date': datetime.datetime(2020, 2, 8, 0, 0), 'total_amount': 313.16},
 {'date': datetime.datetime(2022, 6, 4, 0, 0), 'total_amount': 714.2},
 {'date': datetime.datetime(2015, 11, 4, 0, 0), 'total_amou